In [15]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
from typing import List
import numpy as np
import joblib
from mprompt.modules.fmri_module import SAVE_DIR_FMRI
import imodelsx.util
from mprompt.modules.emb_diff_module import EmbDiffModule
import scipy.stats
from mprompt.methods.m4_evaluate import D5_Validator
import torch.cuda
num_top_test_ngrams = 75
r = (pd.read_pickle('../results/results_fmri.pkl')
    .sort_values(by=['top_score_synthetic'], ascending=False))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
columns = {
    # fmri stuff
    'subject': 'Subject',
    'module_num': 'Voxel',
    'roi_func': 'ROI (functional)',
    'roi_anat': 'ROI (anatomical)',
    'fmri_test_corr': 'Correlation (test)',
    
    # scores
    'top_score_synthetic': 'Synthetic data score',
    'frac_top_ngrams_module_correct': 'Fraction of matching ngrams (module, top-75)',
    # 'frac_top_ngrams_test_correct': 'Fraction of matching ngrams (test, top-75)',
            
    # explanation
    'top_explanation_init_strs': 'Explanation',
        
    # ngrams matching the explanation (used 75 ngrams)
    'top_ngrams_module_correct': 'Matching ngrams (module, top-75)',
    # 'top_ngrams_test_correct': 'Matching ngrams (test, top-75)',

    # all ngrams
    'top_ngrams_module_25': 'Top ngrams (module, top-25)',
    # 'top_ngrams_test_25': 'Top ngrams (test, top-25)',

    # alternative explanations
    'explanation_init_strs': 'Explanation candidates',
}

tab = (
    r
    .sort_values(by=['top_score_synthetic'], ascending=False)
    .filter(columns.keys())
    .rename(columns=columns)
    .round(3)
)
with pd.option_context('display.max_colwidth', -1, 'display.max_rows', 200):
    display(
        tab.head(3)
    )
    tab_join_lists = tab.applymap(lambda x: ' __ '.join(x) if isinstance(x, np.ndarray) or isinstance(x, list) else x)
    tab_join_lists.to_csv('../results/results_fmri.csv', index=False, float_format='%.3f')

,Subject,Voxel,ROI (functional),ROI (anatomical),Correlation (test),Synthetic data score,"Fraction of matching ngrams (module, top-75)",Explanation,"Matching ngrams (module, top-75)","Top ngrams (module, top-25)",Explanation candidates
72,UTS03,43,--,[inferiorparietal],0.611,0.089,0.80,"identity, either through profession, heritage, or location","[date an ornithologist, became a chemist, other egghead phds, alcoholic legal secretary, to art school, spoke passionate russian, at art school, my indian heritage, working at harvard, at christian college, our indian heritage, been a psychoanalyst, waspy jewish, moved to vermont, become a biologist, 'm a chemist, i 'm jewish, a gay speechwriter, my phd, and our jewish, become a lawyer, 're a pagan, she 's jewish, medieval catholic, nineties new york, in art school, was french canadian, hardscrabble new york, 'm jewish, like feeling jewish, as a botanist, father speaking russian, a pastry chef, became a paratrooper, met in college, a molecular geneticist, in business school, marley called zimbabwe, christian university, a psychoanalyst, from rikers island, look very jewish, they were catholic, our jewish policeman, major in psychology, more waspy jewish, that ancient catholic, evangelical christian university, harvard business school, a chemist i, jewish in, was at harvard, degrees in history, baptist university, at culinary school, a gay liberal, becoming a lawyer, puerto rican, 's a chemist, a happy jewish]","[date an ornithologist, became a chemist, other egghead phds, alcoholic legal secretary, to art school, spoke passionate russian, at art school, my indian heritage, pronounce ancient latin, working at harvard, at christian college, our indian heritage, been a psychoanalyst, waspy jewish, moved to vermont, boyish almost biblical, become a biologist, 'm a chemist, i 'm jewish, a gay speechwriter, pennsylvania to beijing, agriculture a manhattan, my phd, and our jewish, become a lawyer]","[identity, either through profession, heritage, or location, education and/or career paths]"
73,UTS03,47,--,[supramarginal],0.551,0.088,0.52,"communication, either through phone calls, doorbells, or knocking","[the doorbell rang, later the doorbell, my phone rang, phone was ringing, my phone rings, our phone rings, neighbor came over, the phone rings, doorbell rang, friend called in, phone rings and, calls me later, she called again, she hung up, somebody then yelled, office knocked on, already called ahead, me her number, calls from friends, partner michael radioed, get her phone, phone rings, they call upstairs, called the neighbor, calls my brother, he called he, phone through colleagues, skype the unboyfriend, phone call from, phone rang, calls me and, called her back, other pager buzzed, give my number, she hangs up, she hanged up, pager buzzed, called me from, messages came pouring]","[the doorbell rang, later the doorbell, my phone rang, bunkmates start talking, phone was ringing, my phone rings, guard spotted us, our phone rings, neighbor came over, the phone rings, doorbell rang, friend called in, phone rings and, room went silent, calls me later, she called again, uh gets remarried, she hung up, somebody then yelled, acquaintance walks in, office knocked on, meet her parents, already called ahead, me her number, find his parents]","[communication, either through phone calls, doorbells, or knocking, communication]"
111,UTS03,29,[IFSFP],"[bankssts, inferiorparietal, middletemporal]",0.512,0.088,0.24,laughter or amusement,"[asked i laughed, she started laughing, smiled i smiled, say just kidding, she laughed and, embarrassment she giggled, she laughed cause, um she laughed, the surprise laughter, all laughed and, n't finished laughing, she laughs then, starte to laugh, i started laughing, were either laughing, fridge i laughed, started to laugh, safe she scoffed]","[asked i laughed, room went silent, said oh okay, she started laughing, said oh ok, five like okay, i stopped midstride, i pro

In [21]:
# display all rows
with pd.option_context('display.max_rows', None):
    display(r.sort_values(by=['top_score_synthetic'])[['subject', 'module_num', 'top_explanation_init_strs', 'top_ngrams_module_correct']])

,subject,module_num,top_explanation_init_strs,top_ngrams_module_correct
12,UTS03,24,a specific year or age,"[in nineteen fifty, of nineteen fifty, in nine..."
4,UTS01,7,movement or action,"[turned and walked, stopped midstride, slowly ..."
35,UTS01,12,repetition,"[talking drinking eating, rejection rejection ..."
62,UTS03,26,movement,"[turned and walked, and darted downstairs, dar..."
69,UTS03,18,repetition,"[bawling bawling, started bawling bawling, scr..."
9,UTS03,55,numbers,"[factor of twenty, hour and twenty, hundred an..."
124,UTS03,19,passing of time,"[the days passed, weeks became months, time we..."
2,UTS03,10,a form of repetition,"[very dark very, smiled i smiled, very moving ..."
47,UTS01,21,relationships,"[felt like brothers, us like family, friends c..."
49,UTS03,35,sound or noise,"[hear boys laughing, i provoked gasps, she sta..."


# Test predictions example

In [ ]:
dsets = joblib.load(join(SAVE_DIR_FMRI, 'stories', 'running_words.jbl'))
neg_dists_list = []
rankcorr_list = []

row = r.iloc[0]
expl = row['top_explanation_init_strs']
dset = dsets[row['subject']]
strs_list = dset['words']
resp = dset['resp'][:, row['module_num']]
neg_dists = row['neg_dists_expl_test']

In [ ]:
# neg_dists = frac_valid
plt.figure(figsize=(7, 6))
plt.plot(neg_dists[:200], resp[:200], '.')
plt.xlabel('Negative distance\n(more negative should correlate with lower response)')
plt.ylabel('Voxel Response')
res = scipy.stats.spearmanr(neg_dists, resp, nan_policy='omit', alternative='greater')
print('rankcorr', res.statistic.round(3))

# Visualize how things relate to each other

In [ ]:
scores = ['fmri_test_corr', 'top_score_synthetic', 'frac_top_ngrams_module_correct']
sns.pairplot(
    r[scores], kind='reg', diag_kind='kde', plot_kws={'scatter_kws': {'alpha': 0.1}}, markers='.',
    height=3, aspect=2
    )
plt.show()